In [1]:
import os
from pathlib import Path
from decimal import ROUND_HALF_UP, Decimal

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [2]:
# I/O Func
def adjusting_price(price, key: str):
    """[Adjusting Close Price]
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """

    def generate_adjusted(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, f"CumulativeAdjustmentFactor{key}"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, f"Adjusted{key}"] = (
            df[f"CumulativeAdjustmentFactor{key}"] * df[key]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df[f"Adjusted{key}"] == 0, f"Adjusted{key}"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, f"Adjusted{key}"] = df.loc[:, f"Adjusted{key}"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted).reset_index(drop=True)

    # price.set_index("Date", inplace=True)
    return price

def adjusting_volume(price, key = "Volume"):
    """[Adjusting Close Price]
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """

    def generate_adjusted(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, f"CumulativeAdjustmentFactor{key}"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, f"Adjusted{key}"] = (
            df[key] / df[f"CumulativeAdjustmentFactor{key}"]  
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df[f"Adjusted{key}"] == 0, f"Adjusted{key}"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, f"Adjusted{key}"] = df.loc[:, f"Adjusted{key}"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted).reset_index(drop=True)

    # price.set_index("Date", inplace=True)
    return price

def read_prices(dir_name: str, securities_code: int = None):
    """[Important: the dateset of 2020/10/1 is lost because of system failer in JPX, see: https://www.jpx.co.jp/corporate/news/news-releases/0060/20201019-01.html]
    
    """
    base_path = Path(f'../input/jpx-tokyo-stock-exchange-prediction/{dir_name}')
    df = pd.read_csv(base_path / 'stock_prices.csv')
    df.loc[: ,"Date"] = pd.to_datetime(df.loc[: ,"Date"], format="%Y-%m-%d")
    df = df[df['Open'].notna()]
    if securities_code:
        df = df[df["SecuritiesCode"] == securities_code]
    return df

def read_stock_list(securities_code: int = None, only_universe: bool = True):
    df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')
    df.loc[: ,"EffectiveDate"] = pd.to_datetime(df.loc[: ,"EffectiveDate"], format="%Y%m%d")
    if only_universe:
        df = df[df['Universe0']]
    if securities_code:
        df = df[df["SecuritiesCode"] == securities_code]
    return df

def read_train_data_by_price(securities_code: int = None, with_supplemental: bool = True):
    """[The train base is price dataset, the other data are joined to prices DF by left join]
    
    """
    def merge_data(prices, stock_list):
        base_df = prices.copy()
        _stock_list = stock_list.copy()
        _stock_list.rename(columns={'Close': 'Close_x'}, inplace=True)
        base_df = base_df.merge(_stock_list, on='SecuritiesCode', how="left")
        return base_df
    
    # origin
    df = merge_data(prices=read_prices(dir_name="train_files", securities_code=securities_code), stock_list=read_stock_list(securities_code=securities_code))
    
    # supplyment
    if with_supplemental:
        supplemental_df = merge_data(prices=read_prices(dir_name="supplemental_files", securities_code=securities_code), stock_list=read_stock_list(securities_code=securities_code))
        df = pd.concat([df, supplemental_df]).reset_index(drop=True)
        
    df = adjusting_price(df, "Close")
    df = adjusting_price(df, "Open")
    df = adjusting_price(df, "High")
    df = adjusting_price(df, "Low")
    df = adjusting_volume(df)
    return df

def write_df(df, filename):
    base_path = Path(f'/kaggle/working')
    df.to_csv(base_path / f'{filename}.csv',index = False)

In [3]:
TOYOTA = 7203

In [4]:
df = read_train_data_by_price(TOYOTA)
df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,...,CumulativeAdjustmentFactorClose,AdjustedClose,CumulativeAdjustmentFactorOpen,AdjustedOpen,CumulativeAdjustmentFactorHigh,AdjustedHigh,CumulativeAdjustmentFactorLow,AdjustedLow,CumulativeAdjustmentFactorVolume,AdjustedVolume
0,20170104_7203,2017-01-04,7203,7010.0,7103.0,6975.0,7097.0,9547600,1.0,NaN,...,0.2,1419.4,0.2,1402.0,0.2,1420.6,0.2,1395.0,0.2,47738000.0
1,20170105_7203,2017-01-05,7203,7090.0,7091.0,7023.0,7049.0,7644000,1.0,NaN,...,0.2,1409.8,0.2,1418.0,0.2,1418.2,0.2,1404.6,0.2,38220000.0
2,20170106_7203,2017-01-06,7203,6840.0,6948.0,6830.0,6930.0,11586100,1.0,NaN,...,0.2,1386.0,0.2,1368.0,0.2,1389.6,0.2,1366.0,0.2,57930500.0
3,20170110_7203,2017-01-10,7203,6922.0,6958.0,6861.0,6861.0,8931100,1.0,NaN,...,0.2,1372.2,0.2,1384.4,0.2,1391.6,0.2,1372.2,0.2,44655500.0
4,20170111_7203,2017-01-11,7203,6899.0,6924.0,6875.0,6912.0,6605300,1.0,NaN,...,0.2,1382.4,0.2,1379.8,0.2,1384.8,0.2,1375.0,0.2,33026500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,20220221_7203,2022-02-21,7203,2148.0,2177.0,2133.0,2173.0,15200700,1.0,NaN,...,1.0,2173.0,1.0,2148.0,1.0,2177.0,1.0,2133.0,1.0,15200700.0
1253,20220222_7203,2022-02-22,7203,2148.0,2151.5,2111.0,2123.5,21140000,1.0,NaN,...,1.0,2123.5,1.0,2148.0,1.0,2151.5,1.0,2111.0,1.0,21140000.0
1254,20220224_7203,2022-02-24,7203,2109.0,2124.0,2080.5,2091.0,29577100,1.0,NaN,...,1.0,2091.0,1.0,2109.0,1.0,2124.0,1.0,2080.5,1.0,29577100.0
1255,20220225_7203,2022-02-25,7203,2085.0,2113.5,2078.5,2104.0,26344300,1.0,NaN,...,1.0,2104.0,1.0,2085.0,1.0,2113.5,1.0,2078.5,1.0,26344300.0


# Featrue

In [5]:
PERIOSDS = [5, 25, 75]
def add_moving_average(df):
    def __add_moving_average(df, key: str):
        for period in PERIOSDS:
            col = f"MovingAverage{key}{period}"
            df[col] = df[key].rolling(period, min_periods=1).mean()
        return df
    df = __add_moving_average(df, "AdjustedClose")
    df = __add_moving_average(df, "AdjustedOpen")
    df = __add_moving_average(df, "AdjustedHigh")
    df = __add_moving_average(df, "AdjustedLow")
    df = __add_moving_average(df, "AdjustedVolume")
    return df

def add_changing_ration(df):
    def __add_changing_ration(df, key: str):
        for period in PERIOSDS:
            col = f"ChangingRatio{key}{period}"
            df[col] = df[key].pct_change(period) * 100
        return df
    df = __add_changing_ration(df, "AdjustedClose")
    df = __add_changing_ration(df, "AdjustedOpen")
    df = __add_changing_ration(df, "AdjustedHigh")
    df = __add_changing_ration(df, "AdjustedLow")
    df = __add_changing_ration(df, "AdjustedVolume")
    return df

def add_historical_vix(df):
    def __add_historical_vix(df, key: str):
        for period in PERIOSDS:
            col = f"HistoricalVIX{key}{period}"
            df[col] = np.log(df[key]).diff().rolling(period).std()
        return df
    df = __add_historical_vix(df, "AdjustedClose")
    df = __add_historical_vix(df, "AdjustedOpen")
    df = __add_historical_vix(df, "AdjustedHigh")
    df = __add_historical_vix(df, "AdjustedLow")
    df = __add_historical_vix(df, "AdjustedVolume")
    return df
    
def add_feature(df):
    # 欠損値処理
    feats = df.copy()
    feats = add_moving_average(feats)
    feats = add_changing_ration(feats)
    feats = add_historical_vix(feats)
    feats = feats.fillna(0)
    # remove 0 depents on max of PERIOSDS
    feats = feats[feats["HistoricalVIXAdjustedClose75"] != 0]
    return feats

feats = add_feature(df)
feats
#write_df(feats, "feats")

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,...,HistoricalVIXAdjustedOpen75,HistoricalVIXAdjustedHigh5,HistoricalVIXAdjustedHigh25,HistoricalVIXAdjustedHigh75,HistoricalVIXAdjustedLow5,HistoricalVIXAdjustedLow25,HistoricalVIXAdjustedLow75,HistoricalVIXAdjustedVolume5,HistoricalVIXAdjustedVolume25,HistoricalVIXAdjustedVolume75
75,20170421_7203,2017-04-21,7203,5820.0,5880.0,5812.0,5855.0,7722600,1.0,0.0,...,0.011694,0.013201,0.010325,0.009253,0.010929,0.010494,0.011044,0.363922,0.339764,0.332889
76,20170424_7203,2017-04-24,7203,5916.0,5928.0,5864.0,5887.0,5533900,1.0,0.0,...,0.011788,0.012207,0.010560,0.009334,0.011062,0.010794,0.011070,0.294381,0.340704,0.334121
77,20170425_7203,2017-04-25,7203,5890.0,6004.0,5887.0,6000.0,7169800,1.0,0.0,...,0.011121,0.012578,0.011026,0.009255,0.010983,0.010861,0.010684,0.293409,0.336381,0.331939
78,20170426_7203,2017-04-26,7203,6091.0,6154.0,6066.0,6154.0,8289400,1.0,0.0,...,0.011750,0.006231,0.011500,0.009745,0.011403,0.011555,0.011280,0.276131,0.305473,0.331049
79,20170427_7203,2017-04-27,7203,6120.0,6135.0,6095.0,6117.0,6931200,1.0,0.0,...,0.011771,0.010096,0.011278,0.009739,0.011037,0.011434,0.011296,0.264362,0.302282,0.329843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,20220221_7203,2022-02-21,7203,2148.0,2177.0,2133.0,2173.0,15200700,1.0,0.0,...,0.020225,0.006786,0.017690,0.017643,0.009167,0.015737,0.015448,0.206159,0.287055,0.338475
1253,20220222_7203,2022-02-22,7203,2148.0,2151.5,2111.0,2123.5,21140000,1.0,0.0,...,0.020137,0.008166,0.017565,0.017554,0.009691,0.015734,0.015305,0.307274,0.279759,0.338762
1254,20220224_7203,2022-02-24,7203,2109.0,2124.0,2080.5,2091.0,29577100,1.0,0.0,...,0.020229,0.007754,0.017195,0.017617,0.005044,0.015571,0.015398,0.317013,0.264311,0.338791
1255,20220225_7203,2022-02-25,7203,2085.0,2113.5,2078.5,2104.0,26344300,1.0,0.0,...,0.020237,0.007693,0.015642,0.017481,0.005698,0.013691,0.015398,0.315975,0.262618,0.326541
